In [1]:
import numpy as np
import pandas as pd
import patsy
from sklearn.ensemble import RandomForestClassifier 

In [2]:
df = pd.read_csv("data/train.csv") 
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


/Users/hqiu/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [4]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [5]:
median_ages = np.zeros((2,3))

for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Gender'] == i) & (df['Pclass'] == j+1)]['Age'].dropna().median()

median_ages

array([[ 35. ,  28. ,  21.5],
       [ 40. ,  30. ,  25. ]])

In [7]:
for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1), 'AgeFill'] = median_ages[i,j]

In [8]:
df[ df['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

,Gender,Pclass,Age,AgeFill
5,1,3,NaN,25.0
17,1,2,NaN,30.0
19,0,3,NaN,21.5
26,1,3,NaN,25.0
28,0,3,NaN,21.5
29,1,3,NaN,25.0
31,0,1,NaN,35.0
32,0,3,NaN,21.5
36,1,3,NaN,25.0
42,1,3,NaN,25.0


In [9]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Gender,AgeFill
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587,29.112424
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,0.477990,13.304424
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.420000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400,0.000000,21.500000
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200,1.000000,26.000000
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000,1.000000,36.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,80.000000


In [10]:
df = df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1) 
df = df.drop(['Age'], axis=1)
df.describe()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Gender,AgeFill
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.523008,0.381594,32.204208,0.647587,29.112424
std,257.353842,0.486592,0.836071,1.102743,0.806057,49.693429,0.477990,13.304424
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.420000
25%,223.500000,0.000000,2.000000,0.000000,0.000000,7.910400,0.000000,21.500000
50%,446.000000,0.000000,3.000000,0.000000,0.000000,14.454200,1.000000,26.000000
75%,668.500000,1.000000,3.000000,1.000000,0.000000,31.000000,1.000000,36.000000
max,891.000000,1.000000,3.000000,8.000000,6.000000,512.329200,1.000000,80.000000


In [11]:
df = df.dropna() 
df.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Gender,AgeFill
0,1,0,3,1,0,7.2500,1,22.0
1,2,1,1,1,0,71.2833,0,38.0
2,3,1,3,0,0,7.9250,0,26.0
3,4,1,1,1,0,53.1000,0,35.0
4,5,0,3,0,0,8.0500,1,35.0


In [12]:
train_data = df.values
print train_data

[[   1.        0.        3.     ...,    7.25      1.       22.    ]
 [   2.        1.        1.     ...,   71.2833    0.       38.    ]
 [   3.        1.        3.     ...,    7.925     0.       26.    ]
 ..., 
 [ 889.        0.        3.     ...,   23.45      0.       21.5   ]
 [ 890.        1.        1.     ...,   30.        1.       26.    ]
 [ 891.        0.        3.     ...,    7.75      1.       32.    ]]


In [13]:
df_predict = pd.read_csv("data/test.csv")
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [14]:
df_predict['Gender'] = df_predict['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [15]:
df_predict['AgeFill'] = df_predict['Age']

for i in range(0, 2):
    for j in range(0, 3):
        df_predict.loc[ (df_predict.Age.isnull()) & (df_predict.Gender == i) & (df_predict.Pclass == j+1), \
                       'AgeFill'] = median_ages[i,j]

In [16]:
df_predict.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender,AgeFill
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1,34.5
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,47.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1,62.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1,27.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,22.0


In [17]:
print df_predict['Fare'].dropna().median()

14.4542


In [18]:
print df_predict[df_predict.Fare.isnull()]

     PassengerId  Pclass                Name   Sex   Age  SibSp  Parch Ticket  \
152         1044       3  Storey, Mr. Thomas  male  60.5      0      0   3701   

     Fare Cabin Embarked  Gender  AgeFill  
152   NaN   NaN        S       1     60.5  


In [19]:
# df_predict.loc[ df_predict['Fare'].isnull(), 'Fare' ] = df_predict['Fare'].dropna().median()
# print df_predict[df_predict.PassengerId == 1044]

if len(df_predict.Fare[ df_predict.Fare.isnull() ]) > 0:
    median_fare = np.zeros(3)
    for f in range(0,3):                                              # loop 0 to 2
        median_fare[f] = df_predict[ df_predict.Pclass == f+1 ]['Fare'].dropna().median()
    for f in range(0,3):                                              # loop 0 to 2
        df_predict.loc[ (df_predict.Fare.isnull()) & (df_predict.Pclass == f+1 ), 'Fare'] = median_fare[f]

In [20]:
df_predict.describe()
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
Gender         418 non-null int64
AgeFill        418 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 42.5+ KB


In [21]:
df_predict = df_predict.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1) 
df_predict = df_predict.drop(['Age'], axis=1)
df_predict.describe()
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Gender         418 non-null int64
AgeFill        418 non-null float64
dtypes: float64(2), int64(5)
memory usage: 22.9 KB


In [22]:
df_predict = df_predict.astype(int)
test_data = df_predict.values
print test_data

[[ 892    3    0 ...,    7    1   34]
 [ 893    3    1 ...,    7    0   47]
 [ 894    2    0 ...,    9    1   62]
 ..., 
 [1307    3    0 ...,    7    1   38]
 [1308    3    0 ...,    8    1   25]
 [1309    3    1 ...,   22    1   25]]


In [23]:
# Import the random forest package
from sklearn.ensemble import RandomForestClassifier 

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 100)

# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train_data[0::,2::],train_data[0::,1])

In [24]:
forest.score(train_data[0::,2::],train_data[0::,1])

0.97979797979797978

In [25]:
# Take the same decision trees and run it on the test data
y_predict = forest.predict(test_data[0::,1::]).astype(int)

In [26]:
print y_predict

[0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 0
 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 1 0 0 1
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 0]


In [27]:
from pandas import DataFrame

output_df = DataFrame({'PassengerId': test_data[0::,0], 'Survived': y_predict})

In [28]:
output_df.to_csv("data/output/randomforest_example.csv", index=False) 